以下の処理を実装してください

- センチメント分析("daigo/bert-base-japanese-sentiment")
- 穴埋め("cl-tohoku/bert-base-japanese-whole-word-masking")
- 要約("tsmatz/mt5_summarize_japanese")
- テキスト生成("rinna/japanese-gpt2-medium")

[Hubで日本語モデルを検索](https://huggingface.co/models?search=japanese)すると、各モデルの詳細が記載されています。

In [ ]:
# !pip install transformers sentencepiece fugashi fugashi[unidic] ipadic
# !python -m unidic download

In [2]:
from transformers import pipeline

analyzer = pipeline("sentiment-analysis", model="daigo/bert-base-japanese-sentiment")
analyzer("私は幸福である。")


[{'label': 'ポジティブ', 'score': 0.9843042492866516}]

In [18]:
unmasker = pipeline("fill-mask", model="cl-tohoku/bert-base-japanese-whole-word-masking")
unmasker("東北大学で[MASK]の研究をしています。", top_k=2)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.0572928860783577,
  'token': 4073,
  'token_str': '医学',
  'sequence': '東北大学 で 医学 の 研究 を し て い ます 。'},
 {'score': 0.05241795629262924,
  'token': 11424,
  'token_str': '考古学',
  'sequence': '東北大学 で 考古学 の 研究 を し て い ます 。'}]

In [3]:
summarizer = pipeline("summarization", model="tsmatz/mt5_summarize_japanese")

text = "人間の知的能力をコンピュータ上で実現する、様々な技術・ソフトウェア・コンピュータシステム。\
    応用例としては、自然言語処理（機械翻訳・かな漢字変換・構文解析・文章要約等）、専門家の推論・判断を模倣するエキスパートシステム、画像データを解析し特定のパターンを検出・抽出する画像認識等がある。\
    1956年にダートマス会議でジョン・マッカーシーにより命名された。\
    現在では、記号処理を用いた知能の記述を主体とする情報処理や研究でのアプローチという意味あいでも使われている。\
    家庭用電気機械器具の制御システムやゲームソフトの思考ルーチンもこう呼ばれることもある。\
    プログラミング言語 LISP による「ELIZA」というカウンセラーを模倣したプログラム（人工無脳）がしばしば引き合いに出されるが、\
    計算機に人間の専門家の役割をさせようという「エキスパートシステム」と呼ばれる研究・情報処理システムの実現は、人間が暗黙に持つ常識の記述が問題となり、当時のその技術では実用化は困難と考えられていた。\
    人工的な知能の実現へのアプローチとしては、「ファジィ理論」や「ニューラルネットワーク」などのようなアプローチも知られているが、従来の人工知能であるGOFAI（Good Old Fashioned AI）との差は記述の記号的明示性にある。\
    その後「サポートベクターマシン」が注目を集めた。また、自らの経験を元に学習を行う強化学習という手法もある。\
    「この宇宙において、知性とは最も強力な形質である（レイ・カーツワイル）」という言葉通り、知性を機械的に表現し実装するということは極めて重要な作業である。\
    画像処理におけるディープラーニングの有用性が競技会で世界的に認知された2012年頃から急速に研究が活発となり、第3次人工知能ブームが到来。2016年から2017年にかけて、\
    ディープラーニングを導入したAIが完全情報ゲームである囲碁などのトップ棋士、さらに不完全情報ゲームであるポーカーの世界トップクラスのプレイヤーも破り、\
    麻雀では「Microsoft Suphx（Super Phoenix）」がオンライン対戦サイト「天鳳」でAIとして初めて十段に到達するなど最先端技術として注目されている。\
    第3次人工知能ブームの主な革命は、自然言語処理、センサーによる画像処理など視覚的側面が特に顕著であるが、社会学、倫理学、技術開発、経済学などの分野にも大きな影響を及ぼしている。"

summarizer(text, max_length=100, min_length=30)

[{'summary_text': '人間の知能をコンピュータ上で実現する、様々な技術・ソフトウェア・コンピュータシステムが、世界各地で注目を集めている。この技術は、人間にとって最も重要な役割を果たしている。'}]

In [26]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")

generator = pipeline('text-generation', model="rinna/japanese-gpt2-medium", tokenizer=tokenizer)

generator("生命、宇宙、そして万物についての究極の疑問の答えは、", max_length = 100, num_return_sequences=3)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '生命、宇宙、そして万物についての究極の疑問の答えは、すべてがひとつに結ばれているという宇宙原理にある。この究極の疑問の答えが何であるのか、それを確かめるには、あらゆるものごとが、無き、無害、無害、無害であるといえない世界が存在することを確信するしか方法はない。「人間は、どんな者でも、自分が何者であるかを知り、それを確信することで、自己(自己)の内なる、自己のための、'},
 {'generated_text': '生命、宇宙、そして万物についての究極の疑問の答えは、この世界を生命が支配していると主張することで得られるのかも知れない。 科学がなければ生命の謎は解けない。また、この世界を科学の力で解明する試みもまた、科学の延長にあるといえるだろう。 宇宙の謎は、宇宙の誕生、地球誕生から現代に至る大きな流れである宇宙の根源的な謎と言える。 この記事へのトラックバック一覧です: 科学と芸術:地球からの'},
 {'generated_text': '生命、宇宙、そして万物についての究極の疑問の答えは、われわれに「神の力」という不思議な力があることを思い出させる。 いまや、われわれは神に導かれるように、その力を活用して生を享受し、生から死までの全ての過程を体験している。 それは、神の存在自体を否定するものではなく、あくまでもそれを受容した経験が神の教えを実践する根拠となっている。 聖書には、「神は宇宙をひとつの存在としておられる」という一つの大きな'}]

# 余力がある人向けの追加演習

### 複数シーケンスの処理
- “I’ve been waiting for a HuggingFace course my whole life.” と “I hate this so much!”に手動でトークン化を適用してください。
- これらをモデルに通して、次のロジットと同じ値が得られることを確認してください。(ロジット = tensor([[-1.5607,  1.6123],[ 4.1692, -3.3464]], grad_fn=<AddmmBackward>))
- 次に、パディングトークンを使ってそれらをバッチ処理し、適切なアテンションマスクを作成します。モデルを通したときに、同じ結果が得られるかどうかチェックしてください(waiting for と hateを含むそれぞれの文が適切に分類されていますか？)


### データセット：MRPCについて
- 訓練集合の要素15と検証集合の要素87を確認してください。それらのラベルは何ですか？
- トレーニングセットの要素15を取り出し、2つの文章を別々に、そしてペアとしてトークン化します。2つの結果の違いは何ですか？

### データセット：GLUE SST-2について
- MRPCと同等な前処理を再現してください。ペアではなく単文で構成されているので少し違いますが、それ以外の部分は同じように見えるはずです。
- GLUEタスクのいずれでも動作する前処理関数を書いてみてください。
- モデルのファインチューニングを行ってみてください。